In [16]:
import sys
import matplotlib.pyplot as plt
import pandas

sys.path.append('/home/jshe/prop-pred/src/data')
from data_utils import *

import torch
import torch.nn as nn
import torch_geometric as torchg
from torch_geometric.loader import DataLoader
from torch_geometric.utils import from_smiles

from gin import GIN

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Data

In [17]:
datadir = '../../data/regress_immuno/'
data = to_namedtuple(to_tensor(load_data(datadir), device))

In [18]:
train_X = torch.concat((
    data.train.embeddings, 
    #data.train.fingerprints
), axis=1)
validation_X = torch.concat((
    data.validation.embeddings, 
    #data.validation.fingerprints
), axis=1)
test_X = torch.concat((
    data.test.embeddings, 
    #ata.test.fingerprints
), axis=1)

In [19]:
train_Y = torch.concat((
    data.train.y, 
    #data.train.y_rdkit, 
), axis=1)[:, 0].unsqueeze(-1)
validation_Y = torch.concat((
    data.validation.y, 
    #data.validation.y_rdkit
), axis=1)[:, 0].unsqueeze(-1)
test_Y = torch.concat((
    data.test.y, 
    #data.test.y_rdkit, 
), axis=1)[:, 0].unsqueeze(-1)

In [20]:
train_graphs = [from_smiles(smiles) for smiles in data.train.smiles]

train_loader = DataLoader(train_graphs, batch_size=32, shuffle=True)

## Model

In [30]:
gin = GIN(in_dim=9, h_dim=64, out_dim=1)

In [35]:
sum(p.numel() for p in gin.parameters())

59073